In [2]:
import pandas as pd
import requests

In [3]:
dates = {'begin': '2023-03-01', 'end': '2023-09-01'}
visits_request = requests.get('https://data-charts-api.hexlet.app/visits', params=dates)
visits_data = visits_request.json()
visits_df = pd.DataFrame(visits_data)

In [7]:
registrations_request = requests.get('https://data-charts-api.hexlet.app/registrations', params=dates)
registrations_data = registrations_request.json()
registrations_df = pd.DataFrame(registrations_data)

In [99]:
visits_df = visits_df[~visits_df['user_agent'].str.contains('bot')]
visits_unique = pd.DataFrame(visits_df.groupby('visit_id').agg('max')['datetime']).reset_index()

In [100]:
unique_visits = pd.merge(visits_df, visits_unique, on=['visit_id', 'datetime'])
unique_visits['date'] = pd.to_datetime(unique_visits['datetime']).dt.date

In [101]:
grouped_unique_visits = unique_visits.drop(['user_agent', 'datetime'], axis=1).groupby(['date', 'platform']).agg('count')
grouped_unique_visits.columns = ['visits']

In [102]:
registrations_df['date'] = pd.to_datetime(registrations_df['datetime']).dt.date
grouped_registrations = registrations_df.drop(['registration_type', 'email', 'datetime'], axis=1).groupby(['date', 'platform']).agg('count')
grouped_registrations.columns = ['registrations']

In [103]:
conversions = pd.merge(grouped_unique_visits, grouped_registrations, on=['date', 'platform'])
conversions['conversion'] = (conversions['registrations'] / conversions['visits'] ).round(3)
conversions = conversions.reset_index()
conversions.columns = ['date_group', 'platform', 'visits', 'registrations', 'conversion']

In [104]:
conversions = conversion.sort_values('date_group')
conversions.to_json('conversion.json')

Шаг 4

In [105]:
ads_df=pd.read_csv('ads.csv')
ads_df['date_group'] = pd.to_datetime(ads_df['date']).dt.date
ads_df = ads_df.drop(['utm_source', 'utm_medium', 'date'], axis=1)

In [106]:
conversions = conversions.drop(['platform', 'conversion'], axis=1)
conversions = conversions.groupby('date_group').agg('sum')

In [108]:
conv_ads = pd.merge(conversions, ads_df,  on=['date_group'], how='left')
conv_ads = conv_ads[['date_group', 'visits', 'registrations', 'cost', 'utm_campaign']]

In [110]:
conv_ads['cost'] = conv_ads['cost'].fillna(0)
conv_ads['utm_campaign'] = conv_ads['utm_campaign'].fillna('none')

In [98]:
conv_ads = conv_ads.sort_values('date_group')
conv_ads.to_json('ads.json')